In [2]:
import os
import dotenv
import pandas as pd
import geopandas as gpd
import pyeuropeana.apis as apis
import pyeuropeana.utils as utils

In [3]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from utils import get_provider_data, aggregate_location_counts

In [4]:
dotenv.load_dotenv()

True

In [5]:
print(os.getenv('EUROPEANA_API_KEY'))

snetrourst


In [15]:
# counts = pd.read_csv('/home/azureuser/cloudfiles/code/Users/sebastien.peytrignet/storytelling-europeana/data/country_counts.csv')
# import ast
# names = [list(ast.literal_eval(elem)[0].values())[0] for elem in counts.iloc[:,0]]
# country_counts = pd.DataFrame({'name': names, 'count': counts.iloc[:,1]})
# country_counts = country_counts.groupby('name').sum().reset_index()
# country_counts.to_csv('/home/azureuser/cloudfiles/code/Users/sebastien.peytrignet/storytelling-europeana/data/country_counts_dedup.csv', index=False)

In [ ]:
# Load world map dataset
world = gpd.read_file("lowres_world.geojson")

In [ ]:
df = get_provider_data("Museum of World Culture", world)

In [ ]:
location_counts = aggregate_location_counts(df)
print(location_counts)